In [2]:
# Trim the audio so that we have 3 sec to train AI

from pydub import AudioSegment 

audio_file = AudioSegment.from_wav("og_audio.wav") 

first_3_seconds = audio_file[:10 * 300] 

first_3_seconds.export("trimmed_audio.wav", format="wav") 

<_io.BufferedRandom name='trimmed_audio.wav'>

In [24]:
from openai import OpenAI
client = OpenAI(api_key='sk-proj-LkOmiWODw7EWRMniR3QKT3BlbkFJJMjOeAa2Ult4sRXSgZql')

audio_file= open("quiet.wav", "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file
)
print(transcription.text)

For my last holiday, I did a trip to Poland and a trip to Scotland. So, I started with a trip to Scotland to visit my friend for his wedding. It was a guy called Adam, I knew, he works for the fund as well. And we moved back to Poland with his girlfriend, who then very quickly became his fiancée. And...


In [19]:
from openai import OpenAI
client = OpenAI(api_key='sk-proj-LkOmiWODw7EWRMniR3QKT3BlbkFJJMjOeAa2Ult4sRXSgZql')

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='In the realm of code, a concept profound,\nRecursion whispers, unique and unbound.\nLike a mirror reflecting its own reflection,\nA function calls itself, a powerful connection.\n\nThrough loops of self, it journeys deep,\nSolving problems complex, in dreams it seeps.\nA dance of echoes, reverberating in time,\nUnraveling mysteries, a poetic rhyme.\n\nWith elegance and grace, it weaves its tale,\nDividing tasks, without fail.\nA fractal pattern, expanding wide,\nInfinite layers, where solutions hide.\n\nThrough branches of function, it travels far,\nIn a recursive journey, like a shooting star.\nAn endless cycle, a programming art,\nRecursion enchants, touching the heart.', role='assistant', function_call=None, tool_calls=None)
